## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

In [54]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV, train_test_split

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    - Número de vezes que, para cada alteração no sinal EMG, tal modificação exceda o limiar pré-definido de 10mV.

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    - Representa características no domínio do tempo baseadas na amplitude do sinal, como o valor absoluto médio, inclinação do valor absoluto médio e variância. Em comparação com os outros algoritmos, o RMS provê um desempenho, em termos de tolerância, mais robusto que outros.

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)

    - Comprimento acumuldo da forma de onda ao longo do segmento de tempo. Está relacionado com a amplitude, frequência e tempo da forma de onda 
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    - Número de vezes que os sinais EMG passam por zero. Esta característica fornece uma estimativa aproximada das propriedades no domínio da frequência. O limiar é de 20mV.

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)


    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    - É a frequência média do espectro. É calculada a partir da soma dos produtos das potências do espectro de potência e da frequência, dividida pela soma total da intensidade do espectro.

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)


    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    - Calculada como a soma dos produtos da amplitude do espectro e da frequência, dividida pela soma total da intensidade do espectro.

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Respostas


1. FMD: é a média das frequências

2. MMNF: é a média das frequências baseado na amplitude do espectro

3. WAMP: é a contagem de quantas amplitude excedem o threshold definido

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [55]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def wamp(time, threshold):
    return np.sum(np.abs(np.diff(time)) > threshold, axis=-1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2


def func_j(w):
    sample_rate = np.arange(1, w.shape[-1]+1) * 200
    return sample_rate / 2 * w.shape[-1]

def fmn(w):
    return np.sum(func_j(w) * PSD(w)) / fmd(w)*2

def mmnf(w):
    return np.sum(func_j(w) * np.abs(w),  axis=-1) / mmdf(w)*2

def wl(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def zc(x):
    return np.count_nonzero(np.diff(np.sign(x), axis=-1) != 0, axis=-1)


**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [56]:
# Carregando dados do "prepare.ipynb"
time = np.load("./datasets/jesse_time.npy")
freq = np.load("./datasets/jesse_freq.npy")


# Extraindo características
data_var = var(time)
data_rms = rms(time)
data_wl = wl(time)
data_wamp = wamp(time, np.median(time))

data_fmd = fmd(freq)
data_mmdf = mmdf(freq)
data_fmn = fmn(freq)
data_mmnf = mmnf(freq)


data_zc = zc(freq)

print(data_wamp.shape, data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape, data_fmn.shape, data_mmnf.shape)

(28, 2, 17) (28, 2, 17) (28, 2, 17) (28, 2, 17) (28, 2, 17) (28, 2, 17) (28, 2, 17)


### Um individuo

In [57]:

features = np.array([data_wamp, data_var, data_rms, data_fmd, data_mmdf, data_fmn, data_wl, data_zc])

# organização das dimensões
features = features.transpose(2, 1, 0, 3)

# # criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1], features.shape[2] * features.shape[3])

print(features.shape)

(56, 136)


*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [58]:
ss_X = StandardScaler().fit_transform(features)

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

Criação do vetor de *labels*

In [59]:
labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]
print('labels_num', len(labels_num))

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[0] / len(labels_num)), axis=-1)
y

labels_num 28


array([0, 0, 1, 1, 2, 2, 3, 3, 2, 2, 3, 3, 3, 3, 1, 1, 0, 0, 3, 3, 0, 0,
       0, 0, 1, 1, 2, 2, 3, 3, 2, 2, 1, 1, 0, 0, 2, 2, 1, 1, 3, 3, 1, 1,
       0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 3, 3])

*Tarefa 5*: Realização da classificação utilizando `SVM`.

#### Usando SELECT K-BEST


In [60]:
# aplicando seleção de características
from sklearn.feature_selection import RFE


max_k = ss_X.shape[1]
start = int(max_k/2)
end = max_k+1

best_acc = 0
best_params = None

for k in range(start, end):
    X_new_ss = SelectKBest(f_regression, k=k).fit_transform(ss_X, y)
    x_train, x_test, y_train, y_test = train_test_split(X_new_ss, y, test_size=0.25, random_state=42)


    param_grid = [
        {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
        {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'rbf', 'poly']},
    ]

    grid = GridSearchCV(svm.SVC(), param_grid, cv=5)
    grid.fit(x_train, y_train)

    # Predição
    y_pred = grid.predict(x_test)
    current_acc = metrics.accuracy_score(y_test, y_pred)

    if current_acc > best_acc:
        best_acc = current_acc
        best_params = grid.get_params(deep=True)


print("Best accuracy: ", best_acc)

Best accuracy:  0.42857142857142855


#### Usando RFE


In [61]:
# aplicando seleção de características
from sklearn.feature_selection import RFE

X_new_ss = RFE(svm.SVC(kernel="linear")).fit_transform(ss_X, y)

x_train, x_test, y_train, y_test = train_test_split(X_new_ss, y, test_size=0.25, random_state=42)


param_grid = [
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'rbf', 'poly']},
]

grid = GridSearchCV(svm.SVC(), param_grid, cv=5)
grid.fit(x_train, y_train)

# Predição
y_pred = grid.predict(x_test)
current_acc = metrics.accuracy_score(y_test, y_pred)

print("Best accuracy: ", current_acc)

Best accuracy:  0.35714285714285715
